In [3]:

from IPython.display import Audio, display
import librosa as li
import soundfile as sf
import numpy as np
import requests
import torch
torch.set_grad_enabled(False)


In [4]:
# let's download an audio file

def download_file(url, file_path):
    """
    Download file from a given URL and save it to the specified file path.
    """
    response = requests.get(url)
    response.raise_for_status()  # This will help to raise an exception if there is an error

    with open(file_path, 'wb') as file:
        file.write(response.content)

# we will use standford ccrma to download an audio file
url = 'https://ccrma.stanford.edu/~jos/wav/gtr-nylon22.wav'
file_path = 'audio.wav'
download_file(url, file_path)

x, sr = li.load('audio.wav',sr=44100)

display(Audio(x, rate=sr))

In [5]:
# I have downloaded rave parameters/weights and build the model
url = 'https://play.forum.ircam.fr/rave-vst-api/get_model/percussion'
# url = 'https://play.forum.ircam.fr/rave-vst-api/get_model/vintage'
# url = 'https://play.forum.ircam.fr/rave-vst-api/get_model/nasa'
# url = 'https://play.forum.ircam.fr/rave-vst-api/get_model/darbouka_onnx'
# url = 'https://play.forum.ircam.fr/rave-vst-api/get_model/VCTK'

file_path = 'model.ts'
download_file(url, file_path)

model = torch.jit.load("model.ts").eval()

In [6]:
x, sr = li.load('audio.wav',sr=44100)
x = torch.from_numpy(x).reshape(1,1,-1)

# we will use encode and decode to the audio with RAVE
z = model.encode(x)
x_hat = model.decode(z).numpy().reshape(-1)

sf.write("model_output.wav", x_hat, sr)
display(Audio(x_hat, rate=sr))

In [7]:
# So now the below code generates audio from random numbers
z = torch.from_numpy(np.random.randn(*z.shape).astype(np.float16))
x_hat = model.decode(z).numpy().reshape(-1)

sf.write("model_output.wav", x_hat, sr)
display(Audio(x_hat, rate=sr))